In [ ]:
"""Data inspection."""

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import numpy as np
import pickle
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
PATH = '/scratch/users/nmiolane/output'

In [2]:
test = pickle.load(open(f'{PATH}/training/train_losses.pkl', 'rb'))
plt.plot(test)

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/users/nmiolane/output/training/train_losses.pkl'

In [14]:
n_epoch = 5
foo = np.load(f'/scratch/users/nmiolane/output/training/imgs/epoch_{n_epoch}_data.npy')
recon = np.load(f'/scratch/users/nmiolane/output/training/imgs/epoch_{n_epoch}_recon.npy')


@interact(id=IntSlider(min=0,max=7,step=1,value=0))
def display_image(id):
    plt.imshow(foo[id][0])
    plt.figure()
    plt.imshow(recon[id][0])

interactive(children=(IntSlider(value=0, description='id', max=7), Output()), _dom_classes=('widget-interact',…